# Classification

In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler , OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

C:\Users\Steve\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Steve\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Steve\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Steve\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
dataset = pd.read_csv("E202-COMP7117-TD01-00 - classification.csv")

In [3]:
#Ganti value dataset yang perlu diganti

for i in range(0,dataset.shape[0]):
    #Free Sulfure Dioxide
    if dataset['free sulfur dioxide'].astype(str).str.contains('High').any() == True:
        dataset["free sulfur dioxide"] = dataset["free sulfur dioxide"].replace('High', 3)
    if dataset['free sulfur dioxide'].astype(str).str.contains('Medium').any() == True:
        dataset["free sulfur dioxide"] = dataset["free sulfur dioxide"].replace('Medium', 2)
    if dataset['free sulfur dioxide'].astype(str).str.contains('Low').any() == True:
        dataset["free sulfur dioxide"] = dataset["free sulfur dioxide"].replace('Low', 1)
    if dataset['free sulfur dioxide'].astype(str).str.contains('Unknown').any() == True:
        dataset["free sulfur dioxide"] = dataset["free sulfur dioxide"].replace('Unknown', 0)
    #Density
    if dataset['density'].astype(str).str.contains('Very High').any() == True:
        dataset["density"] = dataset["density"].replace('Very High', 0)
    if dataset['density'].astype(str).str.contains('High').any() == True:
        dataset["density"] = dataset["density"].replace('High', 3)
    if dataset['density'].astype(str).str.contains('Medium').any() == True:
        dataset["density"] = dataset["density"].replace('Medium', 2)
    if dataset['density'].astype(str).str.contains('Low').any() == True:
        dataset["density"] = dataset["density"].replace('Low', 1)
    #pH
    if dataset['pH'].astype(str).str.contains('Very Basic').any() == True:
        dataset["pH"] = dataset["pH"].replace('Very Basic', 3)
    if dataset['pH'].astype(str).str.contains('Normal').any() == True:
        dataset["pH"] = dataset["pH"].replace('Normal', 2)
    if dataset['pH'].astype(str).str.contains('Very Accidic').any() == True:
        dataset["pH"] = dataset["pH"].replace('Very Accidic', 1)
    if dataset['pH'].astype(str).str.contains('Unknown').any() == True:
        dataset["pH"] = dataset["pH"].replace('Unknown', 0)

Feature Extraction

In [4]:
#Normalize dataset
from sklearn import preprocessing
#print(dataset.shape)
x = dataset[["volatile acidity", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]]
y = dataset["quality"]
x = preprocessing.normalize(x)
#print(normalize_dataset)

In [5]:
#Analisa dengan PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
principalComponent = pca.fit_transform(x)

principalA = pd.DataFrame(data = principalComponent, columns=['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
finalA = pd.concat([principalA, dataset[['quality']]], axis = 1)
print(finalA)

      principal component 1  principal component 2  principal component 3  \
0                 -0.054522              -0.016019              -0.055101   
1                 -0.188804               0.018730              -0.005567   
2                 -0.150314               0.004192              -0.027173   
3                 -0.174261               0.007311               0.020155   
4                 -0.054522              -0.016019              -0.055101   
...                     ...                    ...                    ...   
1594              -0.106580              -0.002811               0.017126   
1595              -0.121920              -0.001330               0.015396   
1596              -0.070662              -0.012096              -0.000192   
1597              -0.109109              -0.002398               0.010591   
1598              -0.079888              -0.017005               0.002301   

      principal component 4 quality  
0                  0.006513  Decent  

In [ ]:
def load_dataset():
    feature = finalA[['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4']]
    target = finalA[['quality']]
    return feature, target

feature, target = load_dataset()

layer = {
    'input' : 4,
    'hidden' : 2,
    'output' : 5
}

weight = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layer['input'], layer['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layer['hidden'], layer['output']])) 
}
bias = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layer['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layer['output']]))
}

feature = OrdinalEncoder().fit_transform(feature)
feature = MinMaxScaler().fit_transform(feature)

target = OneHotEncoder(sparse=False).fit_transform(target)

input_placeholder = tf.placeholder(tf.float32, [None, layer['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layer['output']])

def feed_forward(dataset):
    weightxbias1 = tf.matmul(dataset, weight['input_to_hidden']) + bias['input_to_hidden']
    activation1 = tf.nn.sigmoid(weightxbias1)

    weightxbias2 = tf.matmul(activation1, weight['hidden_to_output']) + bias['hidden_to_output']
    activation2 = tf.nn.sigmoid(weightxbias2)

    return activation2

learning_rate = 0.2
epoch = 5000

output = feed_forward(input_placeholder)
mse = tf.reduce_mean(0.5 * (output - output_placeholder) ** 2) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(mse)
saver = tf.train.Saver()
feature_train, feature_test, target_train, target_test = train_test_split(feature,target,test_size=0.3)
feature2_test, vali_feature, target2_test, vali_target = train_test_split(feature_test,target_test,test_size=0.33)

with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())

    feed_train = {
        input_placeholder: feature_train,
        output_placeholder: target_train
    }
    
    feed_vali = {
        input_placeholder: vali_feature,
        output_placeholder: vali_target
    }
    
    for i in range(epoch):
        sess.run(train, feed_dict=feed_train)
        error = sess.run(mse, feed_dict = feed_train)
        if i % 100 == 0:
            print('Epoch: {}, Current Error: {}'. format(i+100, error))
            
        if i % 500 == 0 and i != 0:
            val_error = sess.run(mse, feed_dict = feed_vali)
            #print('Validation Error: {}'.format(val_error))
            if i == 500 :
                print("Saving....")
                saver.save(sess, "my-model/model.ckpt")
                val_error_prev = val_error
                #print(val_error_prev)
                print("Save Successfully!")
            if i > 500:
                print('Previous Validation Error: {}, Current Validation Error: {}'. format(val_error_prev, val_error))
                if val_error < val_error_prev:
                    print("Saving....")
                    saver.save(sess, "my-model/model.ckpt")
                    val_error_prev = 0
                    val_error_prev = val_error
                    print("Save Successfully!")
                if val_error > val_error_prev:
                    print('Current Validation Error is bigger!')
                       
    feed_test = {
        input_placeholder: feature2_test,
        output_placeholder: target2_test
    }
    
    matches = tf.equal(tf.argmax(output, axis=1),tf.argmax(output_placeholder,axis=1))
    accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))
    print('Accuracy: {}%'.format(sess.run(accuracy*100, feed_dict =feed_test)))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch: 0, Current Error: 0.13448990881443024
Epoch: 100, Current Error: 0.10606434941291809
Epoch: 200, Current Error: 0.09002487361431122
Epoch: 300, Current Error: 0.0809633880853653
Epoch: 400, Current Error: 0.07566513866186142
Epoch: 500, Current Error: 0.07246353477239609
Saving....
